# Yvynation - Earth Engine Analysis Demo
## Interactive MapBiomas Analysis with Indigenous Territories

This notebook demonstrates the complete **Yvynation application** for analyzing land cover changes in Brazilian indigenous territories using:

- **MapBiomas Collection 9** - Land cover classification (1985-2023)
- **Indigenous Territories** - Official territorial boundaries
- **Interactive Maps** - Using geemap and Folium
- **Statistical Analysis** - Area calculations and change detection
- **Visualizations** - Bar charts, temporal trends, comparisons
- **SPOT Data** (Optional) - NDVI-based classification (restricted access)

### Features:
✅ Interactive multi-layer maps  
✅ Area distribution analysis  
✅ Land cover change detection  
✅ Territory-specific filtering  
✅ Temporal trend visualization  
✅ Graceful SPOT data handling  

### Workflow:
1. Initialize Earth Engine
2. Load MapBiomas and territories
3. Create interactive maps
4. Perform area analysis
5. Visualize changes
6. Try SPOT data (if accessible)

## 1. Initialize Earth Engine and Import Libraries

In [ ]:
# Install and import required libraries
import warnings
warnings.filterwarnings('ignore')

import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Import app modules
from app_file import YvynationApp
from analysis import (
    calculate_area_by_class, 
    calculate_land_cover_change,
    compare_areas,
    filter_territories_by_state
)
from visualization import create_mapbiomas_legend
from plots import (
    plot_area_distribution,
    plot_area_comparison,
    plot_area_changes
)
from spot_module import print_spot_info

print("✅ Libraries imported successfully")

In [ ]:
# Authenticate and initialize Earth Engine
try:
    ee.Initialize(project="ee-leandromet")
    print("✅ Earth Engine initialized successfully!")
except Exception as e:
    print(f"Attempting authentication... {e}")
    ee.Authenticate()
    ee.Initialize(project="ee-leandromet")
    print("✅ Earth Engine authenticated and initialized!")

## 2. Initialize Yvynation App and Load Core Data

In [ ]:
# Create and initialize the Yvynation app
app = YvynationApp()

# Load core MapBiomas and territory data
success = app.load_core_data()

if success:
    print("✅ Ready to start analysis!")

## 3. Create Interactive Map with MapBiomas and Territories

Create a basic interactive map showing MapBiomas land cover classification and indigenous territory boundaries.

In [ ]:
# Create basic map centered on Maranhão
# Center coordinates for Maranhão state, Brazil
center = [-45.3, -4.5]
zoom = 7

Map = app.create_basic_map(center=center, zoom=zoom)
Map.to_streamlit(height=600)

print("✅ Map created with MapBiomas 2023 and indigenous territories")

## 4. Display MapBiomas Legend

In [ ]:
create_mapbiomas_legend()

## 5. Analyze Territory: Land Cover Distribution

Calculate and display area statistics for indigenous territories.

In [ ]:
# Define analysis parameters
START_YEAR = 1985
END_YEAR = 2023

# Run analysis
print(f"📊 Running analysis for {START_YEAR}-{END_YEAR}...\n")
analysis_results = app.analyze_territories(start_year=START_YEAR, end_year=END_YEAR)

# Display start year area distribution
print(f"\n📈 {START_YEAR} Land Cover Distribution:\n")
display(analysis_results['area_start'].head(12))

In [ ]:
# Display end year area distribution
print(f"\n📈 {END_YEAR} Land Cover Distribution:\n")
display(analysis_results['area_end'].head(12))

## 6. Visualize Area Distributions: Side-by-Side Comparison

In [ ]:
# Side-by-side area comparison chart
plot_area_comparison(
    analysis_results['area_start'],
    analysis_results['area_end'],
    START_YEAR, 
    END_YEAR,
    top_n=12,
    figsize=(16, 6)
)

## 7. Analyze Land Cover Changes

Calculate and visualize changes in land cover between analysis periods.

In [ ]:
# Display change comparison table
print(f"\n📊 Land Cover Changes ({START_YEAR} to {END_YEAR}):\n")
change_table = analysis_results['comparison'][['Class_Name', 'Area_km2_start', 'Area_km2_end', 'Change_km2', 'Change_pct']].head(15)
change_table.columns = ['Land Cover Class', f'Area {START_YEAR} (km²)', f'Area {END_YEAR} (km²)', 'Change (km²)', 'Change (%)']
display(change_table)

## 8. Visualize Change Diverging Chart

In [ ]:
# Visualize land cover changes (gain/loss)
plot_area_changes(
    analysis_results['comparison'],
    START_YEAR,
    END_YEAR,
    top_n=12,
    figsize=(12, 7)
)

## 9. Create Territory-Specific Analysis Map

Create an interactive map focused on Maranhão territories.

In [ ]:
# Create territory-specific map for Maranhão
print("🗺️  Creating territory map for Maranhão state...\n")

territory_map = app.create_territory_map(
    state_code='MA',  # Maranhão
    years_to_show=[1985, 2000, 2010, 2023],
    center=[-45.3, -4.5],
    zoom=7
)

territory_map.to_streamlit(height=600)

print("✅ Territory map created with multiple time periods")

## 10. SPOT Data Access (Restricted)

⚠️ SPOT 2008 satellite data has restricted access. This section demonstrates graceful handling of restricted datasets.

In [ ]:
# Print SPOT data information
print_spot_info()

In [ ]:
# Attempt to load SPOT data (graceful handling if restricted)
print("\n🛰️  Attempting SPOT data access...\n")
app.load_spot_if_available()

if app.spot_available:
    print("✅ SPOT data is accessible")
    print(f"   Analytic bands: {app.spot_analytic.bandNames().getInfo()}")
    print(f"   Visual bands: {app.spot_visual.bandNames().getInfo()}")
else:
    print("⚠️  SPOT data is not available (restricted access)")
    print("   This is expected - SPOT requires special permissions from Google Earth Engine")
    print("   The app continues to work with MapBiomas data without SPOT")

## 11. Summary and Next Steps

### ✅ What We've Accomplished:

1. **Initialized Yvynation App** - Loaded MapBiomas and indigenous territories
2. **Created Interactive Maps** - Multi-layer maps with land cover and territories
3. **Analyzed Land Cover Changes** - Calculated areas and changes for 1985-2023
4. **Visualized Results** - Bar charts, comparisons, and diverging change charts
5. **Handled Restricted Data** - Gracefully managed SPOT data access restrictions

### 📊 Key Findings:

- Generated land cover distribution for {START_YEAR} and {END_YEAR}
- Identified major changes in territory land cover
- Created interactive maps for territory exploration
- Documented SPOT data requirements for future use

### 🚀 Next Steps:

1. **Filter by Territory Name** - Use `app.create_territory_map(territory_names=['name'])`
2. **Analyze Specific Classes** - Use `get_class_specific_change()` for deforestation tracking
3. **Export Results** - Use `app.export_results()` to save outputs to Cloud Storage
4. **Request SPOT Access** - Contact Google Earth Engine to request restricted SPOT data access
5. **Custom Time Series** - Analyze specific years or multi-year transitions

### 📚 Module Structure:

```
yvynation/
├── app_file.py         → YvynationApp main class
├── analysis.py         → Area, change, filtering functions
├── visualization.py    → Map creation and layers
├── plots.py           → Charts and temporal trends
├── load_data.py       → MapBiomas and territory loading
├── spot_module.py     → Restricted SPOT data handling
└── config.py          → Assets, colors, labels
```

### 🔗 Resources:

- [MapBiomas Brazil](https://mapbiomas.org/)
- [Earth Engine Code Editor](https://code.earthengine.google.com/)
- [Indigenous Territories](https://mapbiomas-territories.org/)
- [SPOT Satellite Data](https://www.airbus.com/en/defence-and-space/space/earth-observation)